# Predección del semestre en cual un estudiante abandona sus estudios de pregrado

<img src="logo_udea.png" alt="drawing" width="100"/>



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas import read_excel
from sklearn.model_selection import train_test_split

## Leyendo los datos

In [2]:
my_sheet = 'SinDuplicadosDepurada' # change it to your sheet name
file_name = 'BDFinal.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name=my_sheet)
df.shape

(6593, 64)

En la siguiente línea se crea la variable `y` categórica.

In [3]:
df['y'] = pd.Categorical(df['NIVEL_PREGRADO'])

In [4]:
pd.crosstab(index=df['y'], columns="Frecuencia")

col_0,Frecuencia
y,
1,4768
2,867
3,455
4,226
5,135
6,73
7,24
8,9
9,7


## Creación de los datos de entrenamiento (train) y de validación (test)
Para particionar los datos originales se usa la función `train_test_split`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df[['PROMEDIO_ULTIMO_SEMESTRE_TERMINADO',
                                                        'VERSION',
                                                        'CREDITOS_ULTIM_SEMEST_MATRIC',
                                                        'IND_POBREZA',
                                                        'Puntaje_total_estan',
                                                        'Puntaje_global',
                                                        'estu_edad']], 
                                                    df['y'], test_size=0.20)

## Ajuste del modelo SVM

Para crear el modelo se usa la función `sklearn.svm.SVC`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](https://scikit-learn.org/0.16/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [6]:
cw = {}
for i in set(y_train):
    cw[i] = np.sum(y_train == i)
print(cw)

{1: 3822, 2: 692, 3: 358, 4: 183, 5: 109, 6: 54, 7: 17, 8: 9, 9: 5, 10: 19, 11: 6}


In [7]:
from sklearn.svm import SVC

model = SVC(class_weight=cw, coef0=0.0, degree=3, gamma='auto', kernel='linear')
model_fit = model.fit(X=X_train, y=y_train)

Para obtener la tabla de confusión hacemos lo siguiente:

In [8]:
t1 = pd.crosstab(index=model.predict(X_train), columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])

print('\n')
print('La tabla de confusión usando train es: \n')
print(t1)

print('\n')
print('La tabla de confusión usando test es: \n')
print(t2)



La tabla de confusión usando train es: 

Membresia     1    2    3    4    5   6   7   8   9   10  11
Prediccion                                                  
1           3822  692  358  183  109  54  17   9   5  19   6


La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           946  175  97  43  26  19   7   2   4


In [9]:
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))

El Accuracy con test es: 0.7246871444823664
El Accuracy con train es: 0.7172100075815011


## Ajuste del modelo KNN

In [10]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=10)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           935  171  96  43  25  18   7   2   4
2            11    4   1   0   1   1   0   0   0


El Accuracy con test es: 0.7250663632916192
El Accuracy con train es: 0.7119029567854435


## Ajuste del modelo ARBOL

In [11]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           729  113  59  23  12  11   2   1   1
2           111   30  16   9   7   3   4   0   1
3            56   19  11   4   0   3   1   0   1
4            21    4   5   5   2   1   0   0   1
5            13    6   5   1   2   1   0   1   0
6             8    1   1   0   1   0   0   0   0
7             3    1   0   1   0   0   0   0   0
8             0    0   0   0   1   0   0   0   0
9             0    0   0   0   1   0   0   0   0
10            1    0   0   0   0   0   0   0   0
11            4    1   0   0   0   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.5890826383623957


## Ajuste del modelo Gradient Tree Boosting

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           923  164  89  36  22  18   6   1   4
2             6    3   3   2   1   0   1   0   0
3             5    5   2   3   3   0   0   1   0
4             3    0   0   2   0   1   0   0   0
5             2    0   0   0   0   0   0   0   0
6             2    0   0   0   0   0   0   0   0
7             2    2   1   0   0   0   0   0   0
8             0    0   1   0   0   0   0   0   0
10            1    0   1   0   0   0   0   0   0
11            2    1   0   0   0   0   0   0   0


El Accuracy con test es: 0.7828972317026924
El Accuracy con train es: 0.7050796057619408


## Ajuste del modelo Forests of randomized trees¶

In [13]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           714  114  60  22  12  10   2   1   1
2           108   31  16   9   5   3   4   1   1
3            60   19  10   5   2   4   1   0   1
4            27    3   6   6   1   1   0   0   1
5            13    7   5   1   2   1   0   0   0
6             9    1   0   0   1   0   0   0   0
7             4    0   0   0   1   0   0   0   0
8             1    0   0   0   1   0   0   0   0
9             4    0   0   0   0   0   0   0   0
10            2    0   0   0   1   0   0   0   0
11            4    0   0   0   0   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.5784685367702805


## Ajuste del modelo Multioutput classification

In [14]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 100, 100), max_iter=300)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           945  174  96  42  26  19   7   2   4
3             1    1   1   1   0   0   0   0   0


El Accuracy con test es: 0.7256351915054987
El Accuracy con train es: 0.7172100075815011


## Ajuste del modelo Stochastic Gradient Descent

In [15]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5000)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   9   10
Prediccion                                      
1           943  174  95  42  26  19   7   2   4
4             3    1   2   1   0   0   0   0   0


El Accuracy con test es: 0.7244975350777398
El Accuracy con train es: 0.7156937073540561


## Un modelo adivinando

In [147]:
respuesta = pd.crosstab(index=df['y'], columns="Frecuencia")

import random

n = X_train.shape[0] # numero observaciones en train
y_al_azar = random.choices(population=respuesta.index.values, weights=respuesta['Frecuencia'], k=n)
y_al_azar = pd.Series(y_al_azar)

print('\n')
t2 = pd.crosstab(index=y_al_azar, columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
accu = t2.values.diagonal().sum() / t2.values.sum()
print('El Accuracy con train es:', accu)



La tabla de confusión usando test es: 

Membresia     1    2    3    4   5   6   7   8   9   10  11
Prediccion                                                 
1           2127  420  241  116  73  38   9   6   4  16   5
2            365   78   43   28  18  11   3   2   0   1   0
3            194   38   26   15   5   4   1   1   0   1   1
4            108   18   12    7   6   1   1   0   1   0   0
5             69   21    5    5   2   0   1   0   0   1   0
6             32    4    4    5   2   0   1   0   0   0   0
7             11    2    0    0   2   0   0   0   0   0   0
8              3    0    1    1   0   0   0   0   0   0   0
9              4    0    1    0   0   0   1   0   0   0   0
10             9    2    3    1   0   0   0   0   0   0   0
11             5    1    0    0   0   0   0   0   0   0   0


El Accuracy con train es: 0.5279283525807212
